# Generating a random sequence of musical notes with Markov Chains

## Import All Required Libraries

- **`pretty_midi`**: A library for handling MIDI files and musical notes in Python. It is used for loading MIDI files, converting note names to numbers, and saving MIDI files.
- **`random`**: Used for generating random choices, such as selecting the next note in the sequence.
- **`os`**: Provides functions to interact with the operating system, such as creating directories.
- **`pygame`**: A library for creating games. It is used to play the MIDI file sounds.
- **`time`**: Used to pause execution while the music is playing.

In [ ]:
import pretty_midi as pm
import random
import os
import pygame
import time

## Create an example note sequence

The created list (*`notes`*) contains all the notes in the C Major scale. The list *`notes`* serves as the pool of note to generate music.

In [ ]:
notes = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

## Define transition matrix for the notes

The dictionary (*`transition_matrix`*) is used to define a transition matrix. This matrix is used to map each note of the C Major scale and the next possible notes that can follow it. 

In [ ]:
transition_matrix = {
    'C': ['D', 'E', 'F', 'G', 'A', 'B'],
    'D': ['C', 'E', 'F', 'G', 'A', 'B'],
    'E': ['C', 'D', 'F', 'G', 'A', 'B'],
    'F': ['C', 'D', 'E', 'G', 'A', 'B'],
    'G': ['C', 'D', 'E', 'F', 'A', 'B'],
    'A': ['C', 'D', 'E', 'F', 'G', 'B'],
    'B': ['C', 'D', 'E', 'F', 'G', 'A'],
}

## Generate music function

The function (*`generate_music`*) is used to create a sequence of musical notes. The function takes as arguments the following:
* *`start_note`* -> The starting (base) note of the sequence;
* *`length`* -> The number of notes to be generated (Default length is 20);
* *`octave`* -> The octave of the notes (Default octave is 4th).

When the function is called, the following occurs:
1. An empty *`music`* list is created to store all the generated notes.
2. The *`current_note`* is set to the *`start_note`*, marking the beginning of the sequence.
3. For the desired number of notes to be generated (*`length`*):
    1. A random next note is chosen from the transition_matrix based on the *`current_note`*. The *`transition_matrix`* maps each note to a list of possible, later notes.
    2. The selected note is converted into a MIDI note number, combining the note name with the octave.
    3. This MIDI note number is appended to the *`music`* list.
    4. The *`current_note`* is updated.
4. The *`music`* list, containing all the predicted/generated notes, is returned. 

In [ ]:
def generate_music(start_note, length=20, octave=4):
    music = [start_note + str(octave)]
    current_note = start_note

    for _ in range(length):
        next_note = random.choice(transition_matrix[current_note])
        note_number = next_note + str(octave)
        music.append(note_number)
        current_note = next_note

    return music

## Generate a music sequence

The *`generated_music`* variable is defined by calling the *`generate_music`* function. as parameters to the function are provided the base note, the number of notes to generate (if different from the default length of 20) and the octave in which the notes to be (if different from the default 4th octave). 

The generated music sequence is printed to the user.


In [ ]:
generated_music = generate_music('C', length=20, octave=2)
print("Generated music sequence:", generated_music)

## Creating MIDI File

The function *`create_midi_file()`* saves the generated music sequence as a MIDI file:
1. *`midi_data`*: Create a new PrettyMIDI object;
2. *`instrument`*: Create a new PrettyMIDI instrument and select its type;
3. *`start_time`* & *`duration`*: Set start time of the note & set its duration;
4. Iterate through the notes of the generated music and append it to the *`instrument`*'s notes;
5. *`instrument`*'s notes are appended to the *`midi_data`* variable;
6. The *`midi_data`* is saved at the provided file save path.

In [ ]:
def create_midi_file(gen_notes, file_path):
    midi_data = pm.PrettyMIDI()
    instrument = pm.Instrument(program=0)  # program=0 is Acoustic Grand Piano
    
    start_time = 0.0
    duration = 0.5
    
    for note_name in gen_notes:
        note_number = pm.note_name_to_number(note_name)
        note = pm.Note(velocity=50,   # Volume Level
                                pitch=note_number, 
                                start=start_time, 
                                end=start_time + duration)
        instrument.notes.append(note)
        
        start_time += duration
    
    midi_data.instruments.append(instrument)
    midi_data.write(file_path)

In [ ]:
# Set folder to save MIDI file in
folder_path = "midi_files_generated"
os.makedirs(folder_path, exist_ok=True)

# Set MIDI file name
midi_filename = "markov_random_notes.mid"
output_midi_file_path = os.path.join(folder_path, midi_filename)

# Create the MIDI file
create_midi_file(generated_music, output_midi_file_path)

## Playing the MIDI file

To play the MIDI file is required to: 
1. Initialize the Pygame library (*`pygame.init()`*);
2. Initialize the Pygame mixer module for playing sound (*`pygame.mixer.init()`*);
3.  Load the generated MIDI file (*`pygame.mixer.music.load(midi_filename)`*);
4. Start playing the MIDI file (*`pygame.mixer.music.play()`*);
5. Wait until the music finishes playing (*`while pygame.mixer.music.get_busy()`*);
6. Quit Pygame (*`pygame.quit()`*).

In [ ]:
def play_midi_file(midi_file):
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load(midi_file)
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        time.sleep(1)

    pygame.quit()


In [ ]:
# Play the generated MIDI file
play_midi_file(output_midi_file_path)